In [1]:
import os
import csv

#reads all the episodes in a particular folder and reutns them in a array consisting of arrays with tuples
def read_Episodes(baseFolder):
    episodes = []
    for filename in sorted(os.listdir(baseFolder)):
        filePath = os.path.join(baseFolder, filename)
        episode = []
        with open(filePath, "r", newline="") as file:
            reader = csv.reader(file)
            for row in reader:
                observation = [float(epi) for epi in row]
                episode.append(observation)
            episodes.append(episode)        
    return episodes

In [2]:
import d3rlpy
import numpy as np
import os

EPIfolder = "../data/episFormula/1. Basic/"

epis = read_Episodes(EPIfolder)

observations = []
actions = []
rewards = []
terminals = []

for epi in epis:
    for observation in epi:
        observations.append(observation[:-2])
        actions.append(observation[-2:-1])
        rewards.append(observation[-1])
    terminals += [0] * (len(epi) - 1) + [1]

observations=np.array(observations)
actions=np.array(actions)
rewards=np.array(rewards)
terminals=np.array(terminals)

print(observations.shape)

dataset = d3rlpy.dataset.MDPDataset(
    action_space=d3rlpy.constants.ActionSpace.CONTINUOUS,
    observations=observations,
    actions=actions,
    rewards=rewards,
    terminals=terminals,
)


# create action scaler
action_scaler = d3rlpy.preprocessing.MinMaxActionScaler()
# if you don't use GPU, set device=None instead.
sac = d3rlpy.algos.SACConfig(action_scaler=action_scaler,batch_size=256).create(device='cuda:0')

# initialize neural networks with the given observation shape and action size.
# this is not necessary when you directly call fit or fit_online method.
#sac.build_with_dataset(dataset)
# calculate metrics with training dataset
td_error_evaluator = d3rlpy.metrics.TDErrorEvaluator(episodes=dataset.episodes)

# evaluate algorithm on the environment
#rewards = env_evaluator(sac, dataset=None)

result = sac.fit(
    dataset,
    n_steps=100000,
    evaluators={
        'td_error': td_error_evaluator,
    }
)

# get first observation from the dataset
observation = observations[0]

# return actions based on the greedy-policy
action = sac.predict(np.expand_dims(observation, axis=0))

# estimate action-values
value = sac.predict_value(np.expand_dims(observation, axis=0), action)

# save full parameters and configurations in a single file.
sac.save('sac.d3')
# load full parameters and build algorithm
sac2 = d3rlpy.load_learnable("sac.d3")

# save full parameters only
sac.save_model('sac.pt')

# save the greedy-policy as TorchScript
sac.save_policy('policy.pt')

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
Users of this version of Gym should be able to simply replace 'import gym' with 'import gymnasium as gym' in the vast majority of cases.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.
c:\Users\NilsWindows\Desktop\research2.0\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\NilsWindows\Desktop\research2.0\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .au

(28149, 7)
2025-11-11 16:01.01
2025-11-11 16:01.01 [info     ] Signatures have been automatically determined. action_signature=Signature(dtype=[dtype('float64')], shape=[(1,)]) observation_signature= [info     ] Signatures have been automatically determined. action_signature=Signature(dtype=[dtype('float64')], shape=[(1,)]) observation_signature=Signature(dtype=[dtype('float64')], shape=[(7,)]) reward_signature=Signature(dtype=[dtype('float64')], shape=[(1,)])
2025-11-11 16:01.01 [info     ] Signature(dtype=[dtype('float64')], shape=[(7,)]) reward_signature=Signature(dtype=[dtype('float64')], shape=[(1,)])
2025-11-11 16:01.01 [info     ] Action size has been automatically determined. action_size=1
Action size has been automatically determined. action_size=1
2025-11-11 16:01.01 [info     ] dataset info                   dataset_info=DatasetInfo(observation_signature=Signature(dtype=[dtype('float64')], shape=[(7,)]), action_signature=Signature(dtype=[dtype('float64')], shape=[(1,)]), rew

Epoch 1/10: 100%|██████████| 10000/10000 [04:23<00:00, 37.99it/s, critic_loss=1.6, actor_loss=0.385, temp=0.457, temp_loss=0.4]


2025-11-11 16:05.42 [info     ] SAC_20251111160105: epoch=1 step=10000 epoch=1 metrics={'time_sample_batch': 0.004788237285614014, 'time_algorithm_update': 0.020591068029403685, 'critic_loss': 1.600145860132575, 'actor_loss': 0.3853520260878839, 'temp': 0.45650069791972636, 'temp_loss': 0.3998913992278976, 'time_step': 0.02582245240211487, 'td_error': 1.6396682746491789} step [info     ] SAC_20251111160105: epoch=1 step=10000 epoch=1 metrics={'time_sample_batch': 0.004788237285614014, 'time_algorithm_update': 0.020591068029403685, 'critic_loss': 1.600145860132575, 'actor_loss': 0.3853520260878839, 'temp': 0.45650069791972636, 'temp_loss': 0.3998913992278976, 'time_step': 0.02582245240211487, 'td_error': 1.6396682746491789} step=10000
=10000
2025-11-11 16:05.42 [info     ] Model parameters are saved to d3rlpy_logs\SAC_20251111160105\model_10000.d3
2025-11-11 16:05.42 [info     ] Model parameters are saved to d3rlpy_logs\SAC_20251111160105\model_10000.d3


Epoch 2/10: 100%|██████████| 10000/10000 [04:33<00:00, 36.52it/s, critic_loss=18.1, actor_loss=1.77, temp=0.475, temp_loss=-0.0147]


...

Epoch 9/10: 100%|██████████| 10000/10000 [04:15<00:00, 39.16it/s, critic_loss=4.12e+5, actor_loss=-64.8, temp=3.42, temp_loss=-0.0313]


2025-11-11 16:42.36 [info     ] SAC_20251111160105: epoch=9 step=90000 epoch=9 metrics={'time_sample_batch': 0.004716932010650635, 'time_algorithm_update': 0.019934752082824705, 'critic_loss': 411652.407670929, 'actor_loss': -64.7956175960064, 'temp': 3.4247985662698746, 'temp_loss': -0.03124606813080609, 'time_step': 0.02508346498012543, 'td_error': 317718.9854068782} step=90000
 [info     ] SAC_20251111160105: epoch=9 step=90000 epoch=9 metrics={'time_sample_batch': 0.004716932010650635, 'time_algorithm_update': 0.019934752082824705, 'critic_loss': 411652.407670929, 'actor_loss': -64.7956175960064, 'temp': 3.4247985662698746, 'temp_loss': -0.03124606813080609, 'time_step': 0.02508346498012543, 'td_error': 317718.9854068782} step=90000
2025-11-11 16:42.36 [info     ] Model parameters are saved to d3rlpy_logs\SAC_20251111160105\model_90000.d3
2025-11-11 16:42.36 [info     ] Model parameters are saved to d3rlpy_logs\SAC_20251111160105\model_90000.d3


Epoch 10/10: 100%|██████████| 10000/10000 [04:09<00:00, 40.06it/s, critic_loss=7.66e+5, actor_loss=-157, temp=3.99, temp_loss=-0.0187]


2025-11-11 16:47.00 [info     ] SAC_20251111160105: epoch=10 step=100000 epoch=10 metrics [info     ] SAC_20251111160105: epoch=10 step=100000 epoch=10 metrics={'time_sample_batch': 0.004655889248847961, 'time_algorithm_update': 0.019438492631912233, 'critic_loss': 765355.9652391296, 'actor_loss': -156.6195544631958, 'temp': 3.9916217998743058, 'temp_loss': -0.018657735065091403, 'time_step': 0.024533433651924133, 'td_error': 476576.7748584585} step=100000
={'time_sample_batch': 0.004655889248847961, 'time_algorithm_update': 0.019438492631912233, 'critic_loss': 765355.9652391296, 'actor_loss': -156.6195544631958, 'temp': 3.9916217998743058, 'temp_loss': -0.018657735065091403, 'time_step': 0.024533433651924133, 'td_error': 476576.7748584585} step=100000
2025-11-11 16:47.00 [info     ] Model parameters are saved to d3rlpy_logs\SAC_20251111160105\model_100000.d3
2025-11-11 16:47.00 [info     ] Model parameters are saved to d3rlpy_logs\SAC_20251111160105\model_100000.d3


c:\Users\NilsWindows\Desktop\research2.0\.venv\lib\site-packages\d3rlpy\torch_utility.py:431: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  chkpt = torch.load(f, map_locatio